In [1]:
import uiautomator2 as u2
import logging

import pymongo_management 
import toolfacebook_lib
from util import log_message 
import xml.etree.ElementTree as ET
from lxml import etree
import asyncio

def connect_and_open_facebook(device_id):
    try:
        # Kết nối thiết bị qua USB
        driver = u2.connect(device_id)
        log_message(f"✅ Kết nối thành công với thiết bị: {device_id}")

        # Mở ứng dụng Facebook
        driver.app_start("com.facebook.katana", ".LoginActivity")
        log_message(f"🚀 Đã mở ứng dụng Facebook trên thiết bị: {device_id}")

        return driver
    except Exception as e:
        log_message(f"❌ Lỗi khi kết nối hoặc mở Facebook: {e}", level=logging.ERROR)
        return None

In [3]:
driver = connect_and_open_facebook("69QGMN8PXWDYPNIF")
toolfacebook_lib.is_screen_changed(driver)

2025-09-17 09:14:55,528 - INFO - ✅ Kết nối thành công với thiết bị: 69QGMN8PXWDYPNIF
2025-09-17 09:14:55,687 - INFO - 🚀 Đã mở ứng dụng Facebook trên thiết bị: 69QGMN8PXWDYPNIF


True

In [ ]:
async def comment_recruitment_post(driver, user_id):
    comment = await pymongo_management.get_comment(user_id)
    log_message(f"{driver.serial} - {comment[0]['message']}", comment[1])
    if 'comment' in comment[0]:
        comment = comment[0]
        link = comment.get("link")
        comment = comment.get("comment", None)
        toolfacebook_lib.redirect_to(driver, link)
        await asyncio.sleep(2)
        driver(text="Bình luận").click()
        await asyncio.sleep(2)
        driver.send_keys(comment, clear=True)
        driver(description="Gửi").click()
        await toolfacebook_lib.back_to_facebook(driver)
await comment_recruitment_post(driver, "test")

In [ ]:
import pymongo_management
import toolfacebook_lib
import xml.etree.ElementTree as ET
from util import log_message
import asyncio
import logging

async def join_group(driver, command_id, user_id, group_link, back_to_facebook = True):
    toolfacebook_lib.redirect_to(driver, "https://facebook.com/" + group_link)
    await asyncio.sleep(5)
    joined_group = driver(textContains="đã tham gia nhóm")
    if joined_group.exists:
        result = await pymongo_management.update_joined_accounts(user_id, group_link)
        log_message(f"{driver.serial} - {result[0]['message']}", result[1])
        await pymongo_management.execute_command(command_id, "Đã thực hiện")
        if back_to_facebook:
            await toolfacebook_lib.back_to_facebook(driver)
        return

    join_button_clicked = False
    join_button = driver(textContains="Tham gia nhóm")
    if join_button.exists:
        join_button.click()
        join_button_clicked = True

        joined_group = driver(textContains="đã tham gia nhóm")
        if joined_group.exists(timeout=5):
            result = await pymongo_management.update_joined_accounts(user_id, group_link)
            log_message(f"{driver.serial} - {result[0]['message']}", result[1])
            await pymongo_management.execute_command(command_id, "Đã thực hiện")
            if back_to_facebook:
                await toolfacebook_lib.back_to_facebook(driver)
            return

    if not await toolfacebook_lib.click_template(driver, "answer_question") and not join_button_clicked:
        log_message(f"{driver.serial} - Nhóm đang chờ duyệt: {group_link}", logging.INFO)
        await pymongo_management.execute_command(command_id, "Đã thực hiện")
        if back_to_facebook:
            await toolfacebook_lib.back_to_facebook(driver)
        return
    await asyncio.sleep(2)
    xml_dump = driver.dump_hierarchy()
    root = ET.fromstring(xml_dump)

    visible_texts = []
    nodes = []

    for node in root.iter():
        visible = node.attrib.get("visible-to-user", "false") == "true"
        text = node.attrib.get("text", "").strip()

        if visible and text:
            visible_texts.append(text)
            nodes.append(node)

    all_questions_answered = True
    if "Trả lời câu hỏi" in visible_texts:
        indexes = [i for i, text in enumerate(visible_texts) if (text == "Viết câu trả lời..." or text == "Bạn có thể chọn nhiều đáp án" or text == "Bạn có thể chọn 1 đáp án" or text == "Gửi") and visible_texts[i-1] != "Viết câu trả lời..."]
        indexes[-1] += 1
        for i in range(len(indexes) - 1):
            answers = []
            for j in range(indexes[i] + 1, indexes[i + 1] - 1):
                answers.append(visible_texts[j])
            await pymongo_management.upload_question(group_link, visible_texts[indexes[i] - 1], visible_texts[indexes[i]], answers)
            answer, log = await pymongo_management.get_answer(group_link, visible_texts[indexes[i] - 1])
            log_message(f"{driver.serial} - {answer['message']}", log)
            if "answer" in answer:
                if visible_texts[indexes[i]] == "Viết câu trả lời...":
                    bounds = nodes[indexes[i]].attrib.get("bounds")
                    bounds = toolfacebook_lib.parse_number(bounds)
                    driver.click((bounds[0] + bounds[2]) // 2, (bounds[1] + bounds[3]) // 2)
                    driver.send_keys(answer['answer'], clear=True)
                    driver.press("back")  # Nhấn nút back để đóng bàn phím
                elif visible_texts[indexes[i]] == "Bạn có thể chọn nhiều đáp án":
                    for i in range(indexes[i] + 1, indexes[i + 1] - 1):
                        if visible_texts[i] in answer['answer']:
                            bounds = nodes[i].attrib.get("bounds")
                            bounds = toolfacebook_lib.parse_number(bounds)
                            driver.click((bounds[0] + bounds[2]) // 2, (bounds[1] + bounds[3]) // 2)
                elif visible_texts[indexes[i]] == "Bạn có thể chọn 1 đáp án":
                    for i in range(indexes[i] + 1, indexes[i + 1] - 1):
                        if visible_texts[i] == answer['answer']:
                            bounds = nodes[i].attrib.get("bounds")
                            bounds = toolfacebook_lib.parse_number(bounds)
                            driver.click((bounds[0] + bounds[2]) // 2, (bounds[1] + bounds[3]) // 2)
                            break
            else:
                all_questions_answered = False
    if all_questions_answered:
        driver(description="Gửi").click()
        result = await pymongo_management.update_temp_joined_accounts(user_id, group_link)
        log_message(f"{driver.serial} - {result[0]['message']}", result[1])
    else:
        driver(resourceId="com.android.systemui:id/back").click()
        driver(resourceId="com.facebook.katana:id/(name removed)", text="THOÁT").click()
    await pymongo_management.execute_command(command_id, "Đã thực hiện")
    if back_to_facebook:
        await toolfacebook_lib.back_to_facebook(driver)
    
async def check_unapproved_groups(driver, user_id):
    unapproved_groups = await pymongo_management.get_unapproved_groups(user_id)
    for group in unapproved_groups:
        await join_group(driver, "", user_id, group['Link'], False)

In [ ]:
import asyncio
import pymongo_management
import toolfacebook_lib
import logging
from util import log_message
from lxml import etree

async def post_to_group(driver, command_id, user_id, group_link, content, files=None):
    if files:
        for file in files:
            await toolfacebook_lib.push_file_to_device(driver.serial, file)
        driver.app_start("com.miui.gallery")
    toolfacebook_lib.redirect_to(driver, "https://facebook.com/" + group_link)
    await asyncio.sleep(2)
    joined_group = driver(textContains="đã tham gia nhóm")
    if not joined_group.exists:
        log_message(f"{driver.serial} - Đăng bài lên nhóm: Chưa tham gia nhóm {group_link}", logging.WARNING)
        await toolfacebook_lib.back_to_facebook(driver)
        await pymongo_management.execute_command(command_id, "Lỗi: Chưa tham gia nhóm")
        return

    post_button = driver(text="Bạn viết gì đi...")
    if post_button.exists:
        post_button.click()
        await asyncio.sleep(2)
        width, height = driver.window_size()
        center_x = width // 2
        center_y = height // 2
        driver.click(center_x, center_y)
        driver.send_keys(content, clear=True)
        if files:
            driver(description="Ảnh/video").click()
            await asyncio.sleep(2)
            driver(description="Chọn nhiều file").click()
            for i in range(len(files)):
                driver.xpath(f'//android.widget.GridView/android.widget.Button[{i + 1}]/android.widget.Button[1]').click()
            driver(text="Tiếp").click()
            await asyncio.sleep(2)
            try:
                driver(scrollable=True).scroll.vert.backward()
            except:
                pass
        driver(text="ĐĂNG").click()
        log_message(f"{driver.serial} - Đăng bài lên nhóm: Đã đăng bài viết vào nhóm {group_link}", logging.INFO)
        log_message(f"{driver.serial} - Đăng bài lên nhóm: Sau 30s sẽ kiểm tra trạng thái bài viết", logging.INFO)
        await pymongo_management.execute_command(command_id, "Đã thực hiện")
        await asyncio.sleep(30)
        await check_post(driver, user_id)
    else:
        log_message(f"{driver.serial} - Đăng bài lên nhóm: Không tìm thấy nút Tạo bài viết", logging.WARNING)
        await pymongo_management.execute_command(command_id, "Lỗi: Không tìm thấy nút Tạo bài viết")
    await toolfacebook_lib.back_to_facebook(driver)
    if files:
        for file in files:
            await toolfacebook_lib.delete_file(driver.serial, file)
async def check_post(driver, user_id):
    posts = await pymongo_management.get_unapproved_posts(user_id)
    if len(posts) == 0:
        return
    for post in posts:
        toolfacebook_lib.redirect_to(driver, "https://facebook.com/" + post['group_link'])
        await asyncio.sleep(1)
        driver(description="Bạn").click()
        while toolfacebook_lib.is_screen_changed(driver):
            xml = driver.dump_hierarchy()
            tree = etree.fromstring(xml.encode("utf-8"))

            # Tìm nút "Thích"
            like_nodes = tree.xpath("//node[@text='Nút Thích. Hãy nhấn đúp và giữ để bày tỏ cảm xúc về bình luận.' or @text='Đã nhấn nút Thích. Nhấn đúp và giữ để thay đổi cảm xúc.']")
            grandparents = []

            for like in like_nodes:
                parent = like.getparent()
                grandparent = parent.getparent() if parent is not None else None

                if grandparent is not None:
                    grandparents.append(grandparent)
            found = False
            for grandparent in grandparents:
                status = "Đang chờ duyệt"
                for child in grandparent.iter():
                    text = child.attrib.get("text", "")
                    if text == post['content']:
                        found = True
                    if text == "Bình luận":
                        status = "Đã đăng thành công"
                if found:
                    break
            if found:
                if post.get('link', "") == "":
                    post_link = await toolfacebook_lib.extract_post_link(driver, grandparent)
                else:
                    post_link = post.get('link', "")
                result = await pymongo_management.update_post_status(post['_id'], status, post_link)
                log_message(f"{driver.serial} - {result[0]['message']}", result[1])
                break
            driver.swipe_ext("up", scale=0.8)
    await asyncio.sleep(1)
    await toolfacebook_lib.back_to_facebook(driver)

In [ ]:
await check_post(driver, "0979068173")

In [ ]:
async def crawl_post_comment(driver, post_link):
    toolfacebook_lib.redirect_to(driver, post_link)
    location_by_type = []
    index = 1
    raw_comments = {}
    temp_parents = {}
    links = {}
    await toolfacebook_lib.change_comment_display_mode(driver)
    driver.swipe_ext("down", scale=0.8)
    await asyncio.sleep(1)
    await toolfacebook_lib.expand_collapse_section(driver)
    while True:
        node = f'//androidx.recyclerview.widget.RecyclerView/android.view.ViewGroup[{index}]/android.view.ViewGroup[1]/android.view.ViewGroup[1]'
        content_node = driver.xpath(f'//androidx.recyclerview.widget.RecyclerView/android.view.ViewGroup[{index}]/android.view.ViewGroup[1]/android.view.ViewGroup[2]')
        if not driver.xpath(node).exists or not content_node.exists:
            driver.swipe_ext("up", scale=0.8)
            index = 1
            await asyncio.sleep(1)
            await toolfacebook_lib.expand_collapse_section(driver)
            await asyncio.sleep(1)
            if not await toolfacebook_lib.is_screen_changed(driver, threshold=0.9):
                break
            continue

        result = await toolfacebook_lib.extract_comment_info(driver, node, raw_comments, links)
        if result:
            await asyncio.sleep(1)
            bounds = content_node.get().attrib.get("bounds")
            bounds = toolfacebook_lib.parse_number(bounds)
            if bounds[0] not in location_by_type and bounds[0] > 0:
                location_by_type.append(bounds[0])
                location_by_type.sort()
            level = len([x for x in location_by_type if x < bounds[0]])
            temp_parents[level] = (result['name'], result['comment'])
            if "time" in result:
                save_result = await pymongo_management.save_post_comment(post_link, result['name'], result['comment'], result['time'], level, temp_parents[level - 1][0] if level > 0 else None, temp_parents[level - 1][1] if level > 0 else None)
                log_message(f"{driver.serial}: {save_result[0]['message']}", save_result[1])
                raw_comments[result['comment']] = result['name']
        index += 1
await crawl_post_comment(driver, "https://www.facebook.com/share/p/19iY4FCKcL/")

In [ ]:
import asyncio
import pymongo_management
import toolfacebook_lib
import logging
from util import log_message

async def post_to_wall(driver, command_id, user_id, content, files=None):
    if files:
        for file in files:
            await toolfacebook_lib.push_file_to_device(driver.serial, file)
        driver.app_start("com.miui.gallery")
    toolfacebook_lib.back_to_facebook(driver)
    post_button = driver(text="Viết bài trên Facebook")
    if post_button.exists(timeout=2):
        post_button.click()
        await asyncio.sleep(2)
        w, h = driver.window_size()
        driver.click(w // 2, h // 4)
        driver.send_keys(content, clear=True)
    
        if files:
            driver(description="Ảnh/video").click()
            await asyncio.sleep(2)
            driver(description="Chọn nhiều file").click()
            for i in range(len(files)):
                driver.xpath(f'//android.widget.GridView/android.widget.Button[{i + 1}]/android.widget.Button[1]').click()
            driver(text="Tiếp").click()
            await asyncio.sleep(2)
            try:
                driver(scrollable=True).scroll.vert.backward()
            except:
                pass
        driver(text="ĐĂNG").click()
        log_message(f"{driver.serial} - Đăng bài lên tường: Đã đăng bài viết lên tường", logging.INFO)
        log_message(f"{driver.serial} - Đăng bài lên tường: Sau 10s sẽ kiểm tra trạng thái bài viết", logging.INFO)
        await asyncio.sleep(10)
        await check_wall_post(driver, user_id)
    else:
        log_message(f"{driver.serial} - Đăng bài lên tường: Không tìm thấy nút Tạo bài viết", logging.WARNING)
    if files:
        for file in files:
            await toolfacebook_lib.delete_file(driver.serial, file)
    await pymongo_management.execute_command(command_id)

async def check_wall_post(driver, user_id):
    posts = await pymongo_management.get_unapproved_wall_posts(user_id)
    if len(posts) == 0:
        return
    contents = {}
    for post in posts:
        contents[post['content']] = post
    await toolfacebook_lib.back_to_facebook(driver)
    await asyncio.sleep(1)
    driver.xpath('//*[@resource-id="android:id/list"]/android.view.ViewGroup[1]/android.view.ViewGroup[1]/android.view.ViewGroup[1]').click()
    start_time = asyncio.get_event_loop().time()
    while toolfacebook_lib.is_screen_changed(driver) and asyncio.get_event_loop().time() - start_time < 300:
        xml = driver.dump_hierarchy()
        tree = etree.fromstring(xml.encode("utf-8"))

        # Tìm nút "Thích"
        like_nodes = tree.xpath("//node[@text='Nút Thích. Hãy nhấn đúp và giữ để bày tỏ cảm xúc về bình luận.' or @text='Đã nhấn nút Thích. Nhấn đúp và giữ để thay đổi cảm xúc.']")
        grandparents = []

        for like in like_nodes:
            parent = like.getparent()
            grandparent = parent.getparent() if parent is not None else None

            if grandparent is not None:
                grandparents.append(grandparent)
        for grandparent in grandparents:
            for child in grandparent.iter():
                text = child.attrib.get("text", "")
                if text in contents:
                    post = contents[text]
                    post_link = await toolfacebook_lib.extract_post_link(driver, grandparent)
                    result = await pymongo_management.update_wall_post_status(post['_id'], "Đã đăng thành công", post_link)
                    log_message(f"{driver.serial} - {result[0]['message']}", result[1])
                    contents.pop(text)
        if len(contents) == 0:
            break
        driver.swipe_ext("up", scale=0.8)
    await asyncio.sleep(1)
    await toolfacebook_lib.back_to_facebook(driver)

In [ ]:
commands = await pymongo_management.get_commands("test")
for command in commands:
    print(command)
    params = command.get("params", {})
    if command['type'] == 'post_to_group':
        await post_to_group(driver, command['_id'], params.get("group_link", ""), params.get("content", ""), params.get("files", []))
    if command['type'] == 'join_group':
        await join_group(driver, command['user_id'], params.get("group_link", ""))
    if command['type'] == 'post_to_wall':
        await post_to_wall(driver, command['_id'], params.get("content", ""), params.get("files", []))

In [4]:
driver.app_start("com.termux")
driver.send_keys("pkg install termux-api\n")

In [10]:
import pymongo_management
client = pymongo_management.get_async_collection("1", "mongodb://tool_duc_la:X%239v%24L%407qT!e2Z%5EmT8%26uP%40rK1%24gY%23bW%5E3s*Nf%25Qz%408Lx!R7v%24X%23pT%5Ee9Z%40mT8%26uP%40rK1%24gY%23bW%5E@123.24.206.25:27017/?authSource=admin", "fb_data")
test_data = await client.find_one()
print(test_data)

OperationFailure: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}